In [1]:
import sys
sys.path.append('..')

In [2]:
from src.data import make_dummies, map_column_values, change_column_type

In [203]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from hyperopt import fmin, tpe
from hyperopt import hp
import numpy as np
from functools import partial
from typing import Dict

In [57]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
def transform_dataset(dataset: DataFrame) -> DataFrame:
    """
    Performs dataset transform to be able to work with this dataset
    later, making dummy variables from categorical ones and cast to 
    correct types. Also removes `customerID` column, cause it not used for prediction
    """
    categorical_columns = ['PhoneService','StreamingTV','gender',
                           'MultipleLines','SeniorCitizen',
                           'Contract','Partner','DeviceProtection',
                           'OnlineSecurity','StreamingMovies',
                           'PaperlessBilling','Dependents',
                           'PaymentMethod','OnlineBackup',
                           'TechSupport','InternetService'
                          ]
    transformed_dataset = make_dummies(dataset, categorical_columns)

    transformed_dataset = change_column_type(transformed_dataset, 'tenure', 'int')
    transformed_dataset = change_column_type(transformed_dataset, 'MonthlyCharges', 'double')
    transformed_dataset = change_column_type(transformed_dataset, 'TotalCharges', 'double')

    transformed_dataset = map_column_values(transformed_dataset, 'Churn',
                                           {'Yes': 1, 'No': 0})
    transformed_dataset = change_column_type(transformed_dataset, 'Churn', 'int')

    transformed_dataset = transformed_dataset.replace('?', None).dropna(how='any')

    transformed_dataset = transformed_dataset.drop('customerID')

    return transformed_dataset


In [6]:
spark = SparkSession.builder.appName('telco_models').getOrCreate()

22/04/26 22:07:02 WARN Utils: Your hostname, nbb-173-74a resolves to a loopback address: 127.0.1.1; using 192.168.1.186 instead (on interface wlp0s20f3)
22/04/26 22:07:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/26 22:07:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/26 22:07:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [39]:
df = spark.read.csv('../data/raw/dataset.csv', header=True)

## Data Transformations

In [95]:
test = spark.read.csv('../data/processed/test.csv/', header=True)
train = spark.read.csv('../data/processed/train.csv/', header=True)

In [96]:
train = transform_dataset(train)
test = transform_dataset(test)

In [97]:
vec = VectorAssembler(inputCols=list(set(train.columns) - set(['Churn'])),
                     outputCol='features')

In [98]:
train = vec.transform(train)
test = vec.transform(test)

In [99]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

In [100]:
train = scaler.fit(train).transform(train)
test = scaler.fit(test).transform(test)

## Models

In [101]:
rf = RandomForestClassifier(labelCol='Churn', featuresCol='features')

In [102]:
model = rf.fit(train)

In [103]:
pred = model.transform(test)

In [104]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='Churn',
    predictionCol='prediction',
    metricName='f1'
)

In [105]:
evaluator.evaluate(pred)

0.7939194637801144

## Pipeline

In [245]:
test = spark.read.csv('../data/processed/test.csv/', header=True)
train = spark.read.csv('../data/processed/train.csv/', header=True)

In [227]:
train.unionByName(test, allowMissingColumns=True).count()

7043

In [246]:
test = transform_dataset(test)
train = transform_dataset(train)

In [229]:
vec = VectorAssembler(inputCols=list(set(train.columns) - set(['Churn'])),
                     outputCol='features')

In [230]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

In [231]:
model = RandomForestClassifier(
    featuresCol='features', labelCol='Churn',
    maxDepth=10, numTrees=30, impurity='gini',
)

In [232]:
pipeline = Pipeline(stages=[vec, scaler, model])

In [233]:
model = pipeline.fit(train)

22/04/27 01:27:56 WARN DAGScheduler: Broadcasting large task binary with size 1446.7 KiB
22/04/27 01:27:57 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB


In [ ]:
model.save()

In [234]:
pred = model.transform(test)

In [235]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='Churn',
    predictionCol='prediction',
    metricName='f1'
)

In [236]:
evaluator.evaluate(pred)

22/04/27 01:28:01 WARN DAGScheduler: Broadcasting large task binary with size 1275.1 KiB


0.828692410087759

## Adding HyperOpt

In [211]:
train = spark.read.csv('../data/processed/train.csv/', header=True)
train = transform_dataset(train)
test = spark.read.csv('../data/processed/test.csv/', header=True)
test = transform_dataset(test)

In [171]:
def random_forest_f1(args) -> float:    
    max_depth, num_trees, impurity = args
    
    vec = VectorAssembler(inputCols=list(set(train.columns) - set(['Churn'])),
                          outputCol='features'
                         )
    rf = RandomForestClassifier(featuresCol='features',
                                labelCol='Churn',
                                maxDepth=max_depth,
                                numTrees=num_trees,
                                impurity=impurity,
                               )
    pred = Pipeline(stages=[vec, rf]).fit(train).transform(test)
    evaluator = MulticlassClassificationEvaluator(
    labelCol='Churn',
    predictionCol='prediction',
    metricName='f1'
    )
    res = evaluator.evaluate(pred)
    return -res
    

In [172]:
space = [
    hp.choice('max_depth', np.arange(5, 20, dtype=int)),
    hp.choice('num_trees', np.arange(20, 100, dtype=int)),
    hp.choice('impurity', ['gini','entropy'])
]

In [173]:
best = fmin(random_forest_f1, space, algo=tpe.suggest, max_evals=100)

  0%|                                   | 0/100 [00:00<?, ?trial/s, best loss=?]

22/04/26 22:57:53 WARN DAGScheduler: Broadcasting large task binary with size 1330.4 KiB
22/04/26 22:57:53 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
22/04/26 22:57:54 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
22/04/26 22:57:54 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
22/04/26 22:57:55 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
22/04/26 22:57:55 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
22/04/26 22:57:56 WARN DAGScheduler: Broadcasting large task binary with size 7.4 MiB
22/04/26 22:57:57 WARN DAGScheduler: Broadcasting large task binary with size 8.2 MiB
22/04/26 22:57:57 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
22/04/26 22:57:58 WARN DAGScheduler: Broadcasting large task binary with size 9.2 MiB


  1%|        | 1/100 [00:07<12:43,  7.71s/trial, best loss: -0.8116950792456098]

22/04/26 22:57:59 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
22/04/26 22:58:01 WARN DAGScheduler: Broadcasting large task binary with size 1403.3 KiB
22/04/26 22:58:01 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
22/04/26 22:58:02 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
22/04/26 22:58:03 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
22/04/26 22:58:04 WARN DAGScheduler: Broadcasting large task binary with size 7.4 MiB
22/04/26 22:58:05 WARN DAGScheduler: Broadcasting large task binary with size 9.6 MiB
22/04/26 22:58:06 WARN DAGScheduler: Broadcasting large task binary with size 11.7 MiB
22/04/26 22:58:08 WARN DAGScheduler: Broadcasting large task binary with size 13.6 MiB
22/04/26 22:58:09 WARN DAGScheduler: Broadcasting large task binary with size 7.7 MiB


  2%|▏       | 2/100 [00:18<15:14,  9.34s/trial, best loss: -0.8116950792456098]

...

22/04/26 23:08:09 WARN DAGScheduler: Broadcasting large task binary with size 1097.5 KiB
22/04/26 23:08:09 WARN DAGScheduler: Broadcasting large task binary with size 1668.9 KiB
22/04/26 23:08:10 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


 99%|███████▉| 99/100 [10:19<00:07,  7.04s/trial, best loss: -0.828692410087759]

22/04/26 23:08:10 WARN DAGScheduler: Broadcasting large task binary with size 1452.6 KiB
22/04/26 23:08:12 WARN DAGScheduler: Broadcasting large task binary with size 1059.1 KiB
22/04/26 23:08:12 WARN DAGScheduler: Broadcasting large task binary with size 1731.8 KiB


100%|███████| 100/100 [10:22<00:00,  6.22s/trial, best loss: -0.828692410087759]


22/04/26 23:08:13 WARN DAGScheduler: Broadcasting large task binary with size 1047.6 KiB
